### Climate Change Impact Analysis

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score

In [14]:
# Load dataset
file_path =  r"C:\Users\user\Documents\flatiron\Phase 5\Capstone-Project---Group-8\index.ipynb"
climate_merged_df = pd.read_csv(file_path,index_col=0)
climate_merged_df.head()

ParserError: Error tokenizing data. C error: Expected 1 fields in line 4, saw 2
